# Numerical project in Python

We will consider the problem of finding the static equilibrium of a chain formed by rigid bars in 2D.
This will be found via an optimization problem with equality constraints.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

## Question 2

Python function that given $z$ as an imput, returns :

$$
c(z) = (c_1 (z) , \ldots , c_{N+1} (z) ) = (l_{i}(x, y)^{2}-L^{2} = \bigl(  (x_{i}-x_{i-1})^{2}+(y_{i}-y_{i-1})^{2}-L^{2} \, , \, i \in [ 1 , \ldots , N+1 ] \bigr)
$$

In [2]:
L=2

In [3]:
def c(z):
    N=np.len(z)/2
    x=z[0:N:1]
    y=z[N:2*N:1]
    c=np.zeros((N+1,1))
    c[0] = (x[0])**2+(y[0])**2 - L**2
    for i in range(1,N)
        l=(x[i]-x[i-1])**2+(y[i]-y[i-1])**2 - L**2
    c[N+1] = (a-x[N-1])**2+(b-y[N-1])**2 - L**2
    return c
    

## Question 3

Python function that given $z$ as an imput, return $\sum_i y_i$.

To do so, we compute a $e = (0 , \ldots , 0 , 1 , \ldots , 1)$ $2n$-array, and we comput the dot product of $e . z$ .

In [4]:
def e(z):
    N=np.len(z)/2
    e=np.zeros(2*N)
    e[N:2*N:1]=np.ones(N)
    return e

In [5]:
def cost(z):
    return np.dot(e(z), z)

## Question 4

Python function that returns the lagrangian of the systeme, given $z$ and $\lambda$ as an imput.

To do so we write the Lagrangian as :

$$
\mathcal{L}(z, \lambda)=e^{\top} z+\lambda^{\top} c(z)
$$

In [6]:
def lag(z,labda):
    e=e(z)
    return np.dot(np.transpose(e),z) + np.dot(np.transpose(labda),c(z))

## Question 5

Python function, that given $z$, $\lambda$ as an imput, yields $\nabla_z \mathcal{L}(z, \lambda)$.

As shown in the report : 
$$
\nabla_{z} \mathcal{L}(z, \lambda)=e+\nabla_{z}^{\top} c(z) \lambda \in \mathbf{R}^{2 N}
$$

where the Jacobian of c is :
$$
\nabla_{z}^{\top} c(z)= 2 
\left[\begin{array}{ccc}
x_1 & -(x_2 - x_1) & 0 & \cdots & 0 \\
0 & (x_2 - x_1) & -(x_3 - x_2) & \cdots & 0 \\
\\
\vdots & & \ddots & \ddots & \vdots \\
& & & & 0\\
0 & \cdots & 0 & (x_N - x_{N-1}) & -(x_N - a)\\
\hline y_1 & -(y_2 - y_1) & 0 & \cdots & 0 \\
0 & (y_2 - y_1) & -(y_3 - y_2) & \cdots & 0 \\
\\
\vdots & & \ddots & \ddots & \vdots \\
& & & & 0\\
0 & \cdots & 0 & (y_N - y_{N-1}) & -(y_N - a)\\
\end{array}\right] \in \mathbf{R}^{2 N \times N+1}
$$

First lets compute this Jacobian matrix :

In [10]:
def cjac(z):
    N=np.len(z)/2
    cjac=np.zeros(2*N, N+1)
    
    cjac[0,0] = 2*z[0]
    cjac[N,0] = 2*z[N]
    cjac[N-1,N] = 2*(z[N-1]-a)
    cjac[2*n-1,N] = 2*(z[N-1]-b)
    
    for j in range(1,N):
        temp = 2*(z[j]-z[j-1])
        cjac[j,j-1] = -temp
        cjac[j,j] = temp
        temp = 2*(z[N+j]-z[N+j-1])
        cjac[j,N+j-1] = -temp
        cjac[j,N+j] = temp
    return cjac

We deduce the gradient of the Lagrangian by the quick vectorial calculation above

In [3]:
def gradlag(z,labda):
    e=e(z)
    cjac = cjac(z)
    return e + np.dot(cjac,labda)

This function also check that the derivatives are correct, by checking that :
$$
\frac{\left\|c(z+\delta)-c(z)-\nabla_{z} c(z) \delta\right\|}{\|\delta\|} \leq 0.01
$$

for a small random perturbation $\delta$, for a given $z$.

In [15]:
def checkcjac(z,delta):
    cpert = c(z+delta)
    c = c(z)
    cjac_delta = np.dot(cjac(z), delta)
    n1 = np.linlag.norm(cpert - c - cjac_delta)
    n2 = np.linlag.norm(delta)
    if n1/n2 < 0.01:
        return true
    return false

## Question 6

Python function, that given $z$, $\lambda$ as an imput, yields $\nabla_{zz} \mathcal{L} (z, \lambda)$.

As shown in the report, the exact calculation of the Hessian of the Lagrangian give us :

$$
\nabla_{zz} \mathcal{L} (z, \lambda) = \begin{bmatrix} A & 0 \\ 0 & A \end{bmatrix}
$$

where :

$$
A =
    2 \begin{pmatrix}
    \lambda_1+\lambda_2 & -\lambda_2 & & & (0)\\
    -\lambda_2 & \lambda_2+\lambda_3 & -\lambda_3 & \\
     & \ddots & \ddots & \ddots \\
      &  & \ddots & \ddots & -\lambda_n \\
     (0) & & & -\lambda_n & \lambda_n+\lambda_{n+1}
    \end{pmatrix}
$$

In [2]:
def hesslag(z, labda):
    N=np.len(z)/2
    A=np.zeros(N, N)
    zero=np.zeros(N, N)
    for i in range(N):
        A[i,i] = labda[i] + labda[i+1]
    for j in range(N-1):
        temp = -labda[i+1]
        A[i,i+1] = temp
        A[i+1,i] = temp
    A = 2*A
    
    hess=np.zeros(2*N,2*N)
    hess[0:N , 0:N] = A
    hess[0:N , N:2*N] = zero
    hess[N:2*N , 0:N] = zero
    hess[N:2*N , N:2*N] = A
    return cjac

We also check that the double derivatives are correct, by checking that :

$$
\frac{\left\| \mathcal{L}(z+\delta)- \mathcal{L}(z) - \nabla_{z} \mathcal{L}(z) \delta - \delta^{T} \nabla_{zz} \mathcal{L}(z,\lambda) \delta \right\|}{\left\|\delta\ \right\|^2} \leq 0.01
$$

for a small random perturbation $\delta$, and for a given $z$ and $\lambda$.

In [5]:
def checkchess(z,delta, labda):
    lagpert = lag(z+delta, labda)
    lag = lag(z, labda)
    gradlag_delta = np.dot(gradlag(z, labda), delta)
    hesslag_delta = (np.transpose(delta)).dot(hesslag(z, labda).dot(delta))
    
    n1 = np.linlag.norm(lagpert - lag - gradlag_delta - hesslag_delta)
    n2 = np.linlag.norm(delta)
    if n1/(n2**2) < 0.1:
        return true
    return false

## Question 7

The problem is not convexe because even if the cost function is convexe, the admissble set is not. Indeed, the admissible set is defined by $c(z) = 0$.

For instance if we take $a = 0, b = \frac{L}{2}$, the minimum energie solution is of the form $ z = (x, y)$ (the chain is hanging down between the two fixations), and the solution $z = (x, -y)$ is also admisible, $c(x,y) = c(x -y)$ (it is the maximum energie solution, the chain is attracted by the top). But the mean vector of this two admissible solutions is $z = (x, 0)$ which is not admisble because it correspond to the a case where the chain is horizontal atached between the point (0,0) and (0, L/2), but its lenght is $L$, which is impossible in a lenght of $\frac{L}{2}$.

This prove taht the problem is non-convex.

## Question 8

